In [35]:
from datetime import datetime

import feedparser
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from boTool import boTool

In [24]:
RssFeed = feedparser.parse(f"https://news.google.com/rss/search?q=iron&hl=en-US&gl=US&ceid=US:en")

In [40]:
cleanTool = boTool();
print(RssFeed.entries[2].description)
cleanTool.clearTag(RssFeed.entries[2].description)

<a href="https://www.dailyunion.com/news/fourth-annual-iron-pour-at-wisconsin-makers-in-whitewater-april/article_0c30f073-83c4-5f0b-8010-252e99a25fe8.html" target="_blank">Fourth annual iron pour at Wisconsin Makers in Whitewater April 13</a>&nbsp;&nbsp;<font color="#6f6f6f">Daily Union</font><p>WHITEWATER — Wisconsin Makers Inc. will host its fourth annual iron pour on Saturday, April 13.</p>


'Fourth annual iron pour at Wisconsin Makers in Whitewater April 13\xa0\xa0Daily UnionWHITEWATER — Wisconsin Makers Inc. will host its fourth annual iron pour on Saturday, April 13.'

In [17]:
# create a dataframe from a word matrix
def wm2df(wm, feat_names):
    
    # create an index for each row
    doc_names = ['Doc{:d}'.format(idx) for idx, _ in enumerate(wm)]
    df = pd.DataFrame(data=wm.toarray(), index=doc_names,
                      columns=feat_names)
    return(df)

In [18]:
def get_rss_desc(*args):
    cleanTool = boTool();
    
    str_search = ",%20".join(*args)
    RssFeed = feedparser.parse(f"https://news.google.com/rss/search?q={str_search}&hl=en-US&gl=US&ceid=US:en")
    
#     titles = [x.title for x in RssFeed.entries]
    descriptions = [cleanTool.clearTag_old(x.description) for x in RssFeed.entries]
    return(descriptions) 


In [19]:
def cal_result(titles):
    vectorizer = TfidfVectorizer(stop_words="english")
    X = vectorizer.fit_transform(titles)
    
    title_df = wm2df(X, vectorizer.get_feature_names())
    sum_df = title_df.sum()
    result_df = sum_df[sum_df > 1]
    return(result_df)

In [20]:
list_desc = get_rss_desc(["iron", "ore", "steel", "vale", "metal"])

In [21]:
list_desc

[b"<div>UBS hikes iron ore price forecasts again\xc2\xa0\xc2\xa0The Australian Financial ReviewVale sees 2019 iron ore shipments up to 75 million mt under prior forecast: CFO\xc2\xa0\xc2\xa0S&amp;P Global PlattsChina's iron ore futures rise as Vale cuts sales estimate\xc2\xa0\xc2\xa0ReutersIron ore prices set to stay high but coal exports face problems\xc2\xa0\xc2\xa0ABC NewsIron Ore Miners Have a Good Day as Vale Provides Output Update\xc2\xa0\xc2\xa0Money Morning Australia<strong>View full coverage on Google News</strong></div>",
 b"<div>Vale's iron ore losses since dam disaster over double planned impact\xc2\xa0\xc2\xa0S&amp;P Global PlattsBrazilian miner Vale said Tuesday the loss of its iron ore fines output on an annualized basis has totaled about 92.8 million mt/year since the dam disaster in late ...</div>",
 b'<div>This Morning in Metals: Shanghai Copper Price Plunges - Steel, Aluminum, Copper, Stainless, Rare Earth, Metal Prices, Forecasting\xc2\xa0\xc2\xa0MetalMinerThis morn

In [13]:
result = cal_result(list_titles)

In [14]:
type(result)
print(result.index)

Index(['2019', 'aluminum', 'analysis', 'australia', 'australian', 'bloomberg',
       'brazil', 'business', 'china', 'chinese', 'concerns', 'copper', 'cuts',
       'dam', 'demand', 'disaster', 'earth', 'eyes', 'fe', 'financial', 'fool',
       'force', 'forecasting', 'fortescue', 'global', 'industry', 'insider',
       'iron', 'market', 'media', 'metal', 'metalminer', 'metals', 'mining',
       'morning', 'motley', 'news', 'nickel', 'ore', 'pellet', 'platts',
       'price', 'prices', 'production', 'rally', 'rare', 'reuters', 'review',
       'rio', 'rises', 'says', 'shanghai', 'stainless', 'steel', 'supply',
       'vale'],
      dtype='object')


In [69]:
remove_list = ['news','cnbc', 'forbes', 'doesn', 'year', 'journal', 'street', 'times', 'report', 'bloomberg', 'post', 'says', 'politico', 'cnn', 'reuters', 'icis', 'data', 'com', 'mueller', ]
clean_result = result[~result.index.isin(remove_list)]

In [70]:
clean_result.sort_values(ascending=False)

economic     2.809308
asia         2.413063
war          2.392355
new          2.351377
stocks       2.145164
deal         2.004392
slowdown     1.769938
japan        1.702037
slowing      1.521186
fears        1.510754
growth       1.502618
wall         1.493500
talks        1.464760
week         1.275216
amid         1.226188
exports      1.219405
fed          1.217362
market       1.121204
york         1.104382
financial    1.088830
world        1.076655
tumble       1.060350
ahead        1.057300
fall         1.032568
recession    1.029573
europe       1.010919
brexit       1.002289
dtype: float64

In [7]:
str_today = datetime.today().strftime('%Y-%m-%d')
result_csv = result.to_csv(f"/Users/carl/Desktop/side_projects/google_rss_analysis/data/{str_today}.csv")

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


FileNotFoundError: [Errno 2] No such file or directory: '/Users/carl/Desktop/side_projects/google_rss_analysis/data/2019-03-26.csv'